# Entanglement Capabilities of Different Ansatzes

In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from analysis_functions import Analysis
import sys
sys.path.append("../")
from vqc.vqc_circuits import UQC

To measure entanglement capability of the different ansatzes, we are going to use the Meyer-Wallach entanglement measure $Q$ of the states generated by it. Here, there are actually three different things we can do:
 - 1) Simply characterize the entanglement capability of the ansatz by measuring Q of the PQC with random weights.
 - 2) Measure Q of the PQC with the final weights and see how much entanglement the PQC has at the end of training.
 - 3) Measure Q every x training steps and see how entanglement evolves during training.

Measure the entanglement of a state:

$$Q(\ket{\psi}) = 2\left(1 - \frac{1}{n}\sum_{j=1}^{n}Tr(\rho_j^2)\right)$$

Measure the entanglement cabaibility of a PQC:

$$Q(\ket{\psi}) = \frac{2}{|S|}\sum_{\theta_i\in S}\left(1 - \frac{1}{n}\sum_{j=1}^{n}Tr(\rho_j^2(\theta_i))\right)$$

## 1) Entanglement Capability of the different ansatzes

In [51]:
def skolik_variational_layer(wires, params):
    [qml.RY(params[i,0], wires[i]) for i in range(len(wires))]
    [qml.RZ(params[i,1], wires[i]) for i in range(len(wires))]

def skolik_entangling_layer(wires):
    [qml.CZ(wires = [i,j]) for i,j in zip(wires, wires[1:])]
    if len(wires) != 2:
        qml.CZ(wires = [wires[0], wires[-1]])

def skolik_data_encoding(wires, data, params):
    [qml.RX(data[i] * params[i], wires[i], id = f"x_{i}") for i in range(len(wires))]

def uqc_layer( wires, data, rotational_weights, input_weights, bias_weights):
    [qml.RZ(np.dot(2 * input_weights[i], data) + bias_weights[i] , wires[i]) for i in range(len(wires))]
    [qml.RY(2 * rotational_weights[i], wires[i]) for i in range(len(wires))]

def schuld_datareup(params, num_qubits, num_layers, data, qubit_to_measure):
    input_weights = params[0]
    rotational_weights = params[1]
    for l in range(num_layers):
        skolik_variational_layer(range(4), rotational_weights[l])
        skolik_entangling_layer(range(4))
        skolik_data_encoding(range(4), data, input_weights[l])
    skolik_variational_layer(range(4), rotational_weights[num_layers-1])
    return qml.density_matrix(qubit_to_measure)

def baseline_datareup(params, num_qubits, num_layers, data, qubit_to_measure):
    input_weights = params[0]
    rotational_weights = params[1]
    skolik_data_encoding(range(4), data,input_weights[0])
    for l in range(num_layers -1):
        skolik_variational_layer(range(4), rotational_weights[l])
        skolik_entangling_layer(range(4))
    skolik_variational_layer(range(4), rotational_weights[num_layers-1])
    return qml.density_matrix(qubit_to_measure)

def uqc(params, num_qubits, num_layers, data, qubit_to_measure):
    rotational_weights = params[0]
    input_weights = params[1]
    bias_weights = params[2]
    for l in range(num_layers):
        uqc_layer(range(num_qubits), data, rotational_weights[l], input_weights[l], bias_weights[l])
        skolik_entangling_layer(range(num_qubits))
    return qml.density_matrix(qubit_to_measure)

In [52]:
dev_4qubits = qml.device("default.qubit", wires = 4)
dev_2qubits = qml.device("default.qubit", wires = 2)

skolik_datareup_circuit = qml.QNode(schuld_datareup, dev_4qubits)
skolik_baseline_circuit = qml.QNode(baseline_datareup, dev_4qubits)
uqc_2qubits_circuit = qml.QNode(uqc, dev_2qubits)
uqc_4qubits_circuit = qml.QNode(uqc, dev_4qubits)

In [53]:
def q(circuit,weights, num_qubits, num_layers, data):
    """
    Returns the Meyer-Wallach measure of entanglement of the state produced by the circuit
    """
    entropy = 0
    for j in range(num_qubits):
        reduced_density_matrix = circuit(weights, num_qubits, num_layers, data, j)
        trace = np.trace(np.matmul(reduced_density_matrix, reduced_density_matrix))
        entropy += trace
    entropy /= num_qubits
    entropy = 1 - entropy
    return 2*entropy

def entangling_capability(circuit, num_qubits, num_layers, circuit_arch, sample = 1024):
    """
    Uses the Meyer-Wallach measure of entanglement to compute the entangling capability of the circuit
    """
    res = np.zeros(sample, dtype = complex)

    for i in range(sample):
        if circuit_arch == "uqc":
            params = [np.random.uniform(low = 0, high = 2*np.pi, size = (num_layers, num_qubits)),
                      np.random.uniform(low = 0, high = 2*np.pi, size = (num_layers, num_qubits, 4)),
                      np.random.uniform(low = 0, high = 2*np.pi, size = (num_layers, num_qubits))]
        else:
            params = [np.random.uniform(low = 0, high = 2*np.pi, size = (num_layers, num_qubits)),
                      np.random.uniform(low = 0, high = 2*np.pi, size = (num_layers, num_qubits, 2))]
            
        data = np.random.uniform(low = -0.05, high = 0.05, size = 4)
        res[i] = q(circuit, params, num_qubits, num_layers, data)
    
    return np.sum(res).real/sample

In [54]:
entangling_capability_uqc_4qubits = entangling_capability(uqc_4qubits_circuit, 4, 5, "uqc")
entangling_capability_uqc_2qubits = entangling_capability(uqc_2qubits_circuit, 2, 5, "uqc")
entangling_capability_skolik_datareup = entangling_capability(skolik_datareup_circuit, 4, 5, "skolik")
entangling_capability_skolik_baseline = entangling_capability(skolik_baseline_circuit, 4, 5, "skolik")

In [55]:
print("Entangling capability of UQC with 4 qubits: ", entangling_capability_uqc_4qubits)
print("Entangling capability of UQC with 2 qubits: ", entangling_capability_uqc_2qubits)
print("Entangling capability of Skolik's data re-uploading circuit: ", entangling_capability_skolik_datareup)
print("Entangling capability of Skolik's baseline circuit: ", entangling_capability_skolik_baseline)

Entangling capability of UQC with 4 qubits:  0.7650856927216944
Entangling capability of UQC with 2 qubits:  0.37320041863333575
Entangling capability of Skolik's data re-uploading circuit:  0.7616876654459404
Entangling capability of Skolik's baseline circuit:  0.7356081891827865
